In [ ]:
import pandas as pd
train=pd.read_csv('../input/60k-stack-overflow-questions-with-quality-rate/train.csv')
train.head()

In [ ]:
train = train[['Title', 'Body','Y']]
train.head()

In [ ]:
train=train[['Body','Y']]
train.head()

In [ ]:
test=pd.read_csv('../input/60k-stack-overflow-questions-with-quality-rate/valid.csv')
test.head()

In [ ]:
test = test[['Title', 'Body','Y']]
test.head()

In [ ]:
test=test[['Body','Y']]
test.head()

In [ ]:
train.shape,test.shape

In [ ]:
train['Y'].unique()

In [ ]:
test['Y'].unique()

In [ ]:
train.Y.value_counts()

In [ ]:
test.Y.value_counts()

In [ ]:
train['encoded_cat'] = train['Y'].astype('category').cat.codes
train.head()

In [ ]:
train.Y.value_counts()

In [ ]:
train.encoded_cat.value_counts()

In [ ]:
test['encoded_cat'] = test['Y'].astype('category').cat.codes
test.encoded_cat.value_counts()

In [ ]:
test.info()

In [ ]:
train.info()

In [ ]:
X_train=list(train['Body'])
X_train[:5]

In [ ]:
X_test=list(test['Body'])
X_test[:5]

In [ ]:
y_train=list(train['encoded_cat'])
y_train[:5]

In [ ]:
y_test=list(test['encoded_cat'])
y_test[:5]

In [ ]:
round(train['Y'].value_counts()*100/len(train),2)

In [ ]:
round(test['Y'].value_counts()*100/len(test),2)

In [ ]:
!pip install transformers

In [ ]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
train_encodings = tokenizer(X_train, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)

In [ ]:
y_train[:5]

In [ ]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
))

In [ ]:
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments

training_args = TFTrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.1,               # strength of weight decay
    logging_steps=100,
    
)

In [ ]:
with training_args.strategy.scope():
    trainer_model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)

trainer = TFTrainer(
    model=trainer_model,                 # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,             # evaluation dataset
)


trainer.train()

In [ ]:
trainer.evaluate(test_dataset)

In [ ]:
trainer.predict(test_dataset)

In [ ]:
from sklearn.metrics import classification_report
target_names = ['LQ_CLOSE','HQ','LQ_EDIT']
print(classification_report(y_test,
                            trainer.predict(test_dataset)[1],target_names=target_names
                           )
     )